In [1]:
import numpy as np
import pandas as pd
# from pyciede2000 import ciede2000
import ast
import math
import tensorflow as tf

import colour
import numpy as np
from itertools import combinations

import sys
sys.path.append('../')

from dccw.multiple_palettes_sorter import * 
from dccw.similarity_measurer import *
from dccw.geo_sorter_helper import *

2024-02-26 01:23:06.847723: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-26 01:23:06.893577: I tensorflow/core/platform/cpu_feature_guard.cc:183] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import sys
sys.path.append('../src')
from full_palette_generation.text_color_model.model_config import Config
from full_palette_generation.text_color_model.input_data_generator import DataGenerator
from full_palette_generation.utils.color_convertor import lab_to_rgb, rgb_to_lab, rgb_to_hex, range0to255

Config_test = Config.copy()

PROJECT_PATH = Config['project_path']
representation = Config['representation']
emb_file = Config['emb_file']
text_model = Config['text_model']
bin_range = Config['bin_range']

In [3]:
# diversity of average pairwise distance
def calculate_diversity(mlm_predict, batch_mlm_mask, origin_x, mask_num):
    batch_mlm_mask = tf.cast(batch_mlm_mask, dtype=tf.int32)
    index = tf.where(batch_mlm_mask == 1)
    # x_predict = tf.math.argmax(mlm_predict, axis=-1) # top1
    x_predict = tf.argsort(mlm_predict, axis=-1, direction='DESCENDING') # topn    
    x_predict = tf.gather_nd(x_predict, index)
    x_real = tf.gather_nd(origin_x, index) # [mask11, mask12, mask21, mask22,...]
    x_predict = x_predict.numpy()
    x_real = x_real.numpy()
    
    diversity_f = []
    diversity_r = []
    unknown = 0
    
    for p in range(0, len(x_predict), mask_num):
        real_lab_list = []
        pred_lab_list = []
        real_rgb_list = []
        pred_rgb_list = []
        unk = 0
        predicts = []
        for m in range(mask_num):
            real_ = dataset.corpus.token_id_to_word_list([x_real[p+m]])[0]
            if '_' in real_:
                real_lab = real_.split('_')
                real_rgb = lab_to_rgb([int(real_lab[0]) * bin_range, int(real_lab[1]) * bin_range, int(real_lab[2]) * bin_range])
                real_rgb_list.append(real_rgb)
                real_lab_list.append(rgb_to_lab(range0to255(real_rgb)))
            else:
                unk = 1
                
            topn = 0
            pred_ = dataset.corpus.token_id_to_word_list([x_predict[p+m][topn]])[0] # use topn prediction
            # while pred_ in predicts:
            #     topn += 1
            #     pred_ = dataset.corpus.token_id_to_word_list([x_predict[p+m][topn]])[0]
            predicts.append(pred_)  
            if '_' in pred_:
                pred_lab = pred_.split('_')
                pred_rgb = lab_to_rgb([int(pred_lab[0]) * bin_range, int(pred_lab[1]) * bin_range, int(pred_lab[2]) * bin_range])
                pred_rgb_list.append(pred_rgb)
                pred_lab_list.append(rgb_to_lab(range0to255(pred_rgb)))
            else:
                unk = 1
        
        if unk == 0:
            distances = [colour.delta_E((int(c1[0])/255*100, int(c1[1])-128, int(c1[2])-128),
                                        (int(c2[0])/255*100, int(c2[1])-128, int(c2[2])-128),
                                        method='cie2000') for c1, c2 in combinations(pred_lab_list, 2)]
            average_ciede2000_distance = np.mean(distances)
            diversity_f.append(average_ciede2000_distance)
            distances = [colour.delta_E((int(c1[0])/255*100, int(c1[1])-128, int(c1[2])-128),
                                        (int(c2[0])/255*100, int(c2[1])-128, int(c2[2])-128),
                                        method='cie2000') for c1, c2 in combinations(real_lab_list, 2)]
            average_ciede2000_distance = np.mean(distances)
            diversity_r.append(average_ciede2000_distance)
        else:
            unknown += unk
    print(unknown)
    return diversity_f, diversity_r

# similarity metric of DCCW
def calculate_similarity_dccw(mlm_predict, batch_mlm_mask, origin_x, mask_num):
    batch_mlm_mask = tf.cast(batch_mlm_mask, dtype=tf.int32)
    index = tf.where(batch_mlm_mask == 1)
    # x_predict = tf.math.argmax(mlm_predict, axis=-1) # top1
    x_predict = tf.argsort(mlm_predict, axis=-1, direction='DESCENDING') # topn    
    x_predict = tf.gather_nd(x_predict, index)
    x_real = tf.gather_nd(origin_x, index) # [mask11, mask12, mask21, mask22,...]
    x_predict = x_predict.numpy()
    x_real = x_real.numpy()
    
    similarity = []
    unknown = 0
    
    for p in range(0, len(x_predict), mask_num):
        real_hex_list = []
        pred_hex_list = []
        real_rgb_list = []
        pred_rgb_list = []
        unk = 0
        predicts = []
        for m in range(mask_num):
            real_ = dataset.corpus.token_id_to_word_list([x_real[p+m]])[0]
            if '_' in real_:
                real_lab = real_.split('_')
                real_rgb = lab_to_rgb([int(real_lab[0]) * bin_range, int(real_lab[1]) * bin_range, int(real_lab[2]) * bin_range])
                real_rgb_list.append(real_rgb)
                real_hex_list.append(rgb_to_hex(range0to255(real_rgb)))
            else:
                unk = 1
                
            topn = 0
            pred_ = dataset.corpus.token_id_to_word_list([x_predict[p+m][topn]])[0] # use topn prediction
            # while pred_ in predicts:
            #     topn += 1
            #     pred_ = dataset.corpus.token_id_to_word_list([x_predict[p+m][topn]])[0]
            predicts.append(pred_)  
            if '_' in pred_:
                pred_lab = pred_.split('_')
                pred_rgb = lab_to_rgb([int(pred_lab[0]) * bin_range, int(pred_lab[1]) * bin_range, int(pred_lab[2]) * bin_range])
                pred_rgb_list.append(pred_rgb)
                pred_hex_list.append(rgb_to_hex(range0to255(pred_rgb)))
            else:
                unk = 1
        
        if unk == 0:
            # print(f'real_rgb: {real_rgb_list}; pred_rgb: {pred_rgb_list}')
            color_palettes_hex_list = [pred_hex_list, real_hex_list]
            palettes = ColorPalettes(
                auto_fetched=False, color_palettes_list=color_palettes_hex_list, is_hex_list=True)
            source_palette = palettes.get_single_palettes_list()[0]
            target_palette = palettes.get_single_palettes_list()[1]

            similarity_measurer = SimilarityMeasurer(source_palette, target_palette, LabDistanceMode.CIEDE2000)
            similarities, comments = similarity_measurer.measure(include_elapsed_time=False)
            dccw = similarities['DynamicClosestColorWarping']
            similarity.append(dccw)
        else:
            unknown += unk
    print(unknown)
    return similarity
    # return similarity/(len(x_predict) - unknown)


In [4]:
Config_test['Corpus_File_Path'] = os.path.join(PROJECT_PATH, f'color/color_corpus_{representation}_test.txt')
Config_test['Text_Input_File_Path'] = os.path.join(PROJECT_PATH, f'text/text_input_test.txt')
Config_test['Text_Input_Emb_File_Path'] = os.path.join(PROJECT_PATH, f'text/{emb_file}/text_input_emb{text_model}_test.txt')

In [6]:
# get mean data for several trained models
train_times = 1
mask_num = 5

total_acc = [0] * mask_num
total_sim = [0] * mask_num
total_div = [0] * mask_num

for i in range(train_times):
    filename = '../data/trained_model/t2p_ca1_mca1_1t_stop30_lr0.0002_dr0.2_0.3__clip_512d_lab_bins_16_0.8_0.5'
    re_model = tf.keras.models.load_model(f"{filename}_{i}")

    for j in range(mask_num-1, mask_num):
        Config_test['Mask_position'] = 'random'
        Config_test['Mask_num'] = j+1
        Config_test['Batch_Size'] = 1018

        dataset = DataGenerator(Config_test)
        test_batch_x, test_batch_mlm_mask, test_batch_mcc_mask, test_origin_x, test_batch_segment, test_batch_padding_mask, test_batch_text_input_embed = dataset[0]
        
        test_mlm_predict, test_sequence_output = re_model((test_batch_x, test_batch_mlm_mask, test_batch_segment, test_batch_text_input_embed), training=False)
        # acc = calculate_full_predict_accuracy(test_mlm_predict, test_batch_mlm_mask, test_origin_x, Config_test['Mask_num'])
        # total_acc[j] += acc
        # print(f'model:{i}, Mask_num:{j+1}, acc:{acc}')
        
        similarity = calculate_similarity_dccw(test_mlm_predict, test_batch_mlm_mask, test_origin_x, Config_test['Mask_num'])
        print(f'model:{i}, Mask_num:{j+1}, sim:{np.mean(similarity)}, {np.std(similarity)}')
    
        diversity_f, diversity_r = calculate_diversity(test_mlm_predict, test_batch_mlm_mask, test_origin_x, Config_test['Mask_num'])
        print(f'model:{i}, Mask_num:{j+1}, div_f:{np.mean(diversity_f)}, {np.std(diversity_f)}, div_r:{np.mean(diversity_r)}, {np.std(diversity_f)}')
        

2024-02-26 01:24:32.698942: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1638] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13761 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:87:00.0, compute capability: 7.5


/home/jovyan/text2palette/full_palette_generation/notebooks/../dccw/similarity_measurer.py:288: RuntimeWarning: invalid value encountered in double_scalars
  t = max(0, min(np.dot(v, n)/np.dot(n, n), 1))


4
model:0, Mask_num:5, sim:28.126109148585293, 12.906993298402083
4
model:0, Mask_num:5, div_f:29.92011445703687, 10.270641938261688, div_r:26.325088135519376, 10.270641938261688
